In [1]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)


In [2]:
tokenizer = LLaMATokenizer.from_pretrained(
    "decapoda-research/llama-7b-hf"
)

In [3]:
def generate_prompt(data_point):
    # desculpe o desastre de formatação, preciso ser rápido
    if data_point["input"]:
        return f"""
A continuación hay una instrucción que describe una tarea, junto con una entrada que proporciona más contexto. Escriba una respuesta que complete adecuadamente la solicitud.
                ### Instrucción:
                {data_point["instruction"]}
                ### Entrada:
                {data_point["input"]}
                ### Respuesta:"""
    else:
        return f"""A continuación hay una instrucción que describe una tarea. Escriba una respuesta que complete adecuadamente la solicitud.
                ### Instrucción:
                {data_point["instruction"]}
                ### Respuesta:"""
    
def tokenize(prompt):
    result = tokenizer(
        prompt,
        return_tensors="pt",
    )
    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt

In [4]:
model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "CristianC/chivito_lora_alpaca_es_7b")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [10]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=4
)

def evaluate(instruction):
    
    inputs = generate_and_tokenize_prompt(instruction)
    
    input_ids = inputs["input_ids"].cuda()
    
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("Respuesta: \n", output.split("### Respuesta:")[1].strip())


In [6]:
test_input = {
    'instruction': 'Escriba un programa en Python que imprima los primeros 10 números de Fibonacci.',
    'input': ''
}

evaluate(test_input)

Respuesta: 
 # Imprimir los primeros 10 números de Fibonacci

def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

print(fibonacci(10))


In [7]:
test_input = {
    'instruction': "Traduce la oracion 'I have no mouth but I must scream' en español.",
    'input': ''
}

evaluate(test_input)

Respuesta: 
 No tengo boca pero tengo que gritar.


In [8]:
test_input = {
    'instruction': "Traduce la oracion 'hello how are you?' en español.",
    'input': ''
}

evaluate(test_input)

Respuesta: 
 Hola, ¿cómo estás?


In [11]:
test_input = {
    'instruction': "Genera un query SQL",
    'input': """Existen las siguientes tablas de Postgres SQL, con las siguientes caracteristicas:
Employee(id,name,department_id)
Department(id,name,address)
Salary_Payments(id,employee_id,amount,date)

Dame un query que liste los nombres de los departamentos los cuales emplearon mas de 20 empleados.
    """
}

evaluate(test_input)

Respuesta: 
 SELECT department.name
FROM department
JOIN salary_payments ON salary_payments.department_id = department.id
GROUP BY department.name
HAVING COUNT(salary_payments.employee_id) > 20;
